# Sports Scheduling


The Sports Scheduling Problem in a sports league season. For example, a sports league has two divisions, and each division has a number of teams. In each season, the matches between the teams of the two divisions need to be arranged to meet the following requirements:

1. A team can only have one match per week.
2. It is stipulated that the matches between teams within the same division must be carried out as late as possible. The league has corresponding incentive mechanisms, which is also the source of the objective function. We define the profit function as $x^2$, that is, the square of the number of weeks.
3. The league stipulates that the teams within a division need to play a certain number of matches, and the teams between the two divisions also need to play a certain number of matches. The number of matches is all determined by the league.
4. The league stipulates that, despite the constraint in 2, there is a limitation: the number of "matches within a division" (that is, the opponents are teams from the same division) in the first half of the season must be no less than one-third of the total number of matches.
5. Two teams cannot play against each other in two consecutive weeks.  

----------

体育联盟赛季比赛安排问题（Sports Scheduling Problem）。比如一个体育联盟有两个赛区（Division），每个赛区都有一些球队。每赛季需要安排两个赛区之间球队的比赛，满足：

1. 每周一个球队只有一场比赛；
2. 规定同一个赛区内的球队比赛必须尽可能晚地进行：联盟有相应的激励机制，这也是目标函数的来源；我们规定收益函数 = $x^2$， 也就是周数的平方。
3. 联盟规定，赛区内的球队之间需要打满一定场，赛区间的两队也要打满一定场；场数均由联盟制定；
4. 联盟规定，尽管有约束2，但限制：赛季前半段的“赛区内比赛（也就是对手是同赛区的队伍）”必须不少于总比赛数的1/3.
5. 两队不能在连续的两周内比赛；



In [6]:
from collections import namedtuple
import pandas as pd
import seaborn as sns
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [43]:

TEAM_DIV1 = ["Baltimore Ravens","Cincinnati Bengals", "Cleveland Browns","Pittsburgh Steelers","Houston Texans",
                "Indianapolis Colts","Jacksonville Jaguars","Tennessee Titans","Buffalo Bills","Miami Dolphins",
                "New England Patriots","New York Jets","Denver Broncos","Kansas City Chiefs","Oakland Raiders",
                "San Diego Chargers"]

TEAM_DIV2 = ["Chicago Bears","Detroit Lions","Green Bay Packers","Minnesota Vikings","Atlanta Falcons",
                "Carolina Panthers","New Orleans Saints","Tampa Bay Buccaneers","Dallas Cowboys","New York Giants",
                "Philadelphia Eagles","Washington Redskins","Arizona Cardinals","San Francisco 49ers",
                "Seattle Seahawks","St. Louis Rams"]

teams = TEAM_DIV1 + TEAM_DIV2



Match = namedtuple("Matches", ["team1", "team2", "is_divisional"])

# is_divisional: 是否是同一个赛区的，1表示yes，0表示no

nbs = (16, 3, 2)
# 每个赛区有16个球队，intra-division 两两之间比赛3场，inter-division 两两比赛2场；
nb_teams_in_division, nb_intra_divisional, nb_inter_divisional = nbs

nb_weeks = nb_inter_divisional * nb_teams_in_division + (nb_teams_in_division - 1) * nb_intra_divisional

print("{0} games, {1} intradivisional, {2} interdivisional"
          .format(nb_weeks, (nb_teams_in_division - 1) * nb_intra_divisional,
                  nb_teams_in_division * nb_inter_divisional))

first_half_weeks = range(nb_weeks // 2 + 1)
nb_first_half_games = nb_weeks // 3 

# 前三分之一赛季

team_range = range(len(teams))

matches = [Match(teams[t1], teams[t2], 1 if (t2 < nb_teams_in_division or t1 >= nb_teams_in_division) else 0)
               for t1 in team_range for t2 in team_range if t1 < t2]

# for match in matches:
#     print(match)

nb_play = {m: nb_intra_divisional if m.is_divisional == 1 else nb_inter_divisional for m in matches}


m = gp.Model("Sports Scheduling")

print(len(matches))
print(nb_weeks)

plays = m.addMVar((len(matches), nb_weeks), vtype = GRB.BINARY, name = [[f"{match.team1}_{match.team2}_{week + 1}" for week in range(nb_weeks)] for match in matches])


for idx, match in enumerate(matches):
    m.addConstr(gp.quicksum(plays[idx, w] for w in range(nb_weeks)) == nb_play[match] , name = f"constr_{match.team1}_{match.team2}_sum")

# 限制每两个队之间比赛的场次是给定场次

for week in range(nb_weeks):
    for team in teams:
        m.addConstr(gp.quicksum(plays[idx, week] for idx, match in enumerate(matches) if ( match.team1 == team or match.team2 == team )) == 1)

# 限制每周每个队伍只能比赛一次

for w in range(nb_weeks):
    if w + 1 < nb_weeks:
        for idx, _ in enumerate(matches): 
            m.addConstr(plays[idx, w] + plays[idx, w + 1] <= 1)
# # 限制同一个队伍之间不能连续两周比赛


for t in teams:
    m.addConstr(gp.quicksum(plays[idx, week] for week in first_half_weeks for idx, m in enumerate(matches) if m.is_divisional == 1 and (m.team1 == t or m.team2 == t)) >= nb_first_half_games)

# 赛季前半段的“赛区内比赛（也就是对手是同赛区的队伍）”必须不少于总比赛数的1/3.



goal = gp.quicksum(plays[idx, week] * week * week for week in range(nb_weeks) for idx , m in enumerate(matches) if m.is_divisional)

m.setObjective(goal, GRB.MAXIMIZE)

m.optimize()

print(f"Num of Binary Variables: {m.NumBinVars}")
print(f"Num of All Variables: {m.NumVars}")
print(f"Num of All Constrs: {m.NumConstrs}")

if m.status == GRB.Status.OPTIMAL:
    print(f"obj = {m.objVal} ")

else:
    m.computeIIS()
    m.write("model_infeasible.ilp")


77 games, 45 intradivisional, 32 interdivisional
496
77
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40688 rows, 38192 columns and 208688 nonzeros
Model fingerprint: 0xc001d2b8
Variable types: 0 continuous, 38192 integer (38192 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.18s
Presolved: 40688 rows, 38192 columns, 208688 nonzeros
Variable types: 0 continuous, 38192 integer (38192 binary)

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 1.716960e+06, 17589 iterations, 0.89 seconds (1.67 work units)
Total elapsed time = 11.37s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It

In [45]:
all_vars = m.getVars()
values = m.getAttr("X", all_vars)
names = m.getAttr("VarName", all_vars)

for name, val in zip(names, values):
    # print(f"{name} = {val}")
    if val != 0:
        result = name.split("_")
        print(f"{result[0]} vs {result[1]} at week {result[2]}")

Baltimore Ravens vs Cincinnati Bengals at week 17
Baltimore Ravens vs Cincinnati Bengals at week 58
Baltimore Ravens vs Cincinnati Bengals at week 71
Baltimore Ravens vs Cleveland Browns at week 19
Baltimore Ravens vs Cleveland Browns at week 30
Baltimore Ravens vs Cleveland Browns at week 66
Baltimore Ravens vs Pittsburgh Steelers at week 37
Baltimore Ravens vs Pittsburgh Steelers at week 65
Baltimore Ravens vs Pittsburgh Steelers at week 76
Baltimore Ravens vs Houston Texans at week 33
Baltimore Ravens vs Houston Texans at week 72
Baltimore Ravens vs Houston Texans at week 74
Baltimore Ravens vs Indianapolis Colts at week 25
Baltimore Ravens vs Indianapolis Colts at week 69
Baltimore Ravens vs Indianapolis Colts at week 77
Baltimore Ravens vs Jacksonville Jaguars at week 16
Baltimore Ravens vs Jacksonville Jaguars at week 62
Baltimore Ravens vs Jacksonville Jaguars at week 67
Baltimore Ravens vs Tennessee Titans at week 23
Baltimore Ravens vs Tennessee Titans at week 32
Baltimore Rav